In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

from net2 import Generator2, Discriminator2
from data.mpii import MPIIDataset
from utils.vis import draw_keypoints

In [15]:
checkpoint = torch.load('out/out_27.05.2021_09:53/1999_model.pt')

In [16]:
gen = Generator2().to('cuda')
gen.load_state_dict(checkpoint['g_state_dict'])

RuntimeError: Error(s) in loading state_dict for Generator2:
	Unexpected key(s) in state_dict: "downtail.9.weight", "downtail.9.bias", "downtail.10.weight", "downtail.10.bias", "downtail.10.running_mean", "downtail.10.running_var", "downtail.10.num_batches_tracked". 
	size mismatch for head1.0.weight: copying a param with shape torch.Size([16, 32]) from checkpoint, the shape in current model is torch.Size([32, 16]).
	size mismatch for head1.0.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for head1.1.weight: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for head1.1.bias: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for head1.1.running_mean: copying a param with shape torch.Size([16]) from checkpoint, the shape in current model is torch.Size([32]).
	s

In [ ]:
d = Discriminator()
d.load_state_dict(checkpoint['d_state_dict'])

In [ ]:
dataset = MPIIDataset('../data/mpii.json')

In [ ]:
dataset.cat_names

In [ ]:
category = dataset.cat_names[:19]
print(category[11])

In [ ]:
label = dataset.one_hot[:19]
label = Variable(label.type(torch.cuda.LongTensor))

In [ ]:
noise = Variable(torch.cuda.FloatTensor(np.random.normal(0, 1, (19, 32))), requires_grad=False)

In [ ]:
out = gen(noise, label).cpu().detach().numpy()

In [ ]:
i = 0
for j in out:
    print(category[i])
    i += 1
    draw_keypoints(j)

In [ ]:
draw_keypoints([[ 0.1688,  0.7625],
        [ 0.2437,  0.6864],
        [ 0.1220,  1.3411],
        [ 0.5487,  0.8860],
        [ 0.1804,  0.1332],
        [ 0.4840,  0.3490],
        [-0.0175,  0.1754],
        [ 0.9080,  0.4905],
        [ 0.6180,  0.5396],
        [ 1.3200,  0.0510],
        [ 0.7457,  1.4503],
        [ 0.6359,  0.1261],
        [ 0.2172, -0.0303],
        [-0.0024,  0.5620],
        [ 0.5872,  1.1088],
        [ 1.1933,  1.8701]])

In [ ]:
draw_keypoints(dataset[232][0])

In [ ]:
label = dataset.one_hot[0]
for i in range(len(dataset)):
    if (np.array_equal(dataset.one_hot.numpy()[18], dataset[i][1].numpy())):
        draw_keypoints(dataset[i][0])